In [16]:
import seaborn as sns

df = sns.load_dataset('titanic')

display(df.head())
# 컬럼에 대한 내용을 확인
# survived : 생존여부(0 : 사망, 1 : 생존)
# pclass : 등급 (1 : 1등급, 2 : 2등급, 3 : 3등급)
# sex : 성별
# age : 나이
# sibsp : sibling(형제) & spause(배우자)
# parch : parent(부모) & children(자식)
# fare : 요금
# embarked : S, C, Q (승선 위치)

print(df.shape)  # (891, 15)

df.info()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


(891, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [17]:
# Deck이라는 컬럼은 결측치가 많은 것 같아요!
# 일단 결측치를 컬럼별로 확인해 보아요!

df.isna().sum(axis=0) # 많이 사용되는 코드
# 결측치 갯수 확인
# 가장 대표적인 결측치 확인 방법

# 만약 데이터 프레임에 컬럼이 많다면
# 다른 방법으로 확인해야 되요

df.isna().sum(axis=0)[df.isna().sum() > 0]

df.isna().sum(axis=0)[df.isna().any()]

age            177
embarked         2
deck           688
embark_town      2
dtype: int64

In [20]:
# deck은 결측치의 비율이 상당히 높아요!
thresh_df = df.dropna(axis=1,
                      thresh=500,
                      inplace=False) # NaN이 500개 이상인 것만 컬럼 지우기

display(thresh_df)


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,Cherbourg,yes,True


In [ ]:
# 이번에는 행을 지우기
# how= 'any' 해당 row에 NaN이 하나라도 있으면 해당 행을 삭제
# 아래처럼 하면 모든 컬럼에 대해서 NaN을 다 체크
# subset을 이용하며 특정 컬럼에 대해서만 NaN을 체크해요
new_df = thresh_df.dropna(axis=0, how='any')
print(new_df.shape) # (712, 14)

new_df = thresh_df.dropna(subset=['age'],
                          axis=0,
                          how='any')
print(new_df.shape) # (714, 14)

(712, 14)
(714, 14)


In [ ]:
import seaborn as sns

df = sns.load_dataset('titanic')

display(df.head(3))

# 항상 모든 NaN을 삭제할 수는 없어요!
# 일반적으로는 Imputation(대체)해서 사용해야 해요!
# 1. 전통적인 집계함수를 이용해서 처리 - 평균, 중앙값, 최대, 최소, 최빈
# 2. 머신러닝 기법을 이용해서 NaN값을 예측해서 사용

# age안에 177개의 결측치가 있어요
# 가장 쉽게 생각할 수 있는 것은 전체 age의 평균을 구해서
# age의 NaN을 대체

mean_age = df['age'].mean()
print(mean_age) # 29.69911764705882

df['age'] = df['age'].fillna(mean_age) # fillna() NaN을 찾아서 채우는 함수
# fillna()는 기억해야 해요!

# 결측치를 살펴봤더니 ..embarked에도 2개가 있어요
# 그런데 이 embarked는 승선위치라서 문자로 되어 있고, 평균을 낼 수 없어요
# 빈도가 높은 값을 찾아서 채워주는게 좋지 않을까 해요! (최빈값)
print(df['embarked'][825:830]) # NaN 확인
most_freq = df['embarked'].value_counts().idxmax()

df['embarked'] = df['embarked'].fillna(most_freq)
print(df['embarked'][825:830])

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


29.69911764705882
825      Q
826      S
827      C
828      Q
829    NaN
Name: embarked, dtype: object
825    Q
826    S
827    C
828    Q
829    S
Name: embarked, dtype: object


In [43]:
import seaborn as sns

df = sns.load_dataset('titanic')

display(df.head(3))

# 평균같이 수치계산이 힘든건 꼭 최빈값을 이용해야 하나요?
# 사람도 그렇지만... 데이터도 유사한 것 끼리 뭉쳐있는 특성이 있어요!

# ffill : 바로 앞 행에 있는 값으로 치환
# bfill : 바로 뒤 행에 있는 값으로 치환
df['embarked'] = df['embarked'].fillna(method='ffill') # 앞으로 안쓰게 되는 코드

# Forward Fill (앞의 값으로 채움)
df['embarked'] = df['embarked'].ffill()
print(df['embarked'][825:830]) # NaN 확인


# Backward Fill (뒤의 값으로 채움)
df['embarked'] = df['embarked'].bfill()
print(df['embarked'][825:830]) # NaN 확인

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


825    Q
826    S
827    C
828    Q
829    Q
Name: embarked, dtype: object
825    Q
826    S
827    C
828    Q
829    Q
Name: embarked, dtype: object


/tmp/ipykernel_1907/3645164496.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['embarked'] = df['embarked'].fillna(method='ffill') # 앞으로 안쓰게 되는 코드


In [ ]:
# 데이터 전처리 쪽에서 그 다음에 나오는 내용은
# 당연히 이상치 처리에요!
# 이상치는 이상한 값이예요. 나올수 없는 값이나... 잘못 된 값
# 전체 데이터의 분포와 확연히 동떨어진 값

In [ ]:
# 데이터 중복처리
# 하나의 데이터셋에서 동일한 관측치값이 2개 이상 있을 때
# 중복데이터라고 하고 일반적으로 삭제해야 해요!

# Pandas에는 이 중복 데이터를 처리할 수 있는 함수가 2개 있어요
import pandas as pd

df = pd.DataFrame({'c1' : ['a', 'a', 'b', 'a', 'b'],
                   'c2' : [1, 1, 1, 2, 2],
                   'c3' : [1, 1, 2, 2, 2]})

display(df)
'''
    c1	c2	c3
0	a	1	1
1	a	1	1
2	b	1	2
3	a	2	2
4	b	2	2
'''

df.duplicated() # DataFrame의 각 행들이 중복된 행인지 True, False로 알려줘요

df['c2'].duplicated()

# mask를 이용해서 boolean indexing을 이용해서 처리할 수도 있어요!
# 편하게 중복을 제거하는 함수도 같이 제공
df2 = df.drop_duplicates()
display(df2)
'''
    c1	c2	c3
0	a	1	1
2	b	1	2
3	a	2	2
4	b	2	2
'''

df3 = df.drop_duplicates(subset=['c2', 'c3'])
display(df3)
'''
    c1	c2	c3
0	a	1	1
2	b	1	2
3	a	2	2
'''

,c1,c2,c3
0,a,1,1
1,a,1,1
2,b,1,2
3,a,2,2
4,b,2,2


,c1,c2,c3
0,a,1,1
2,b,1,2
3,a,2,2
4,b,2,2


,c1,c2,c3
0,a,1,1
2,b,1,2
3,a,2,2


In [ ]:
# 데이터 타입의 변환 
import numpy as np
import pandas as pd

df = pd.read_csv('./data/auto-mpg.csv',
                 header=None)

df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower',
              'weight', 'acceleration', 'model year', 'origin', 'name']

df.info()
'''
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   name          398 non-null    object 
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB
'''

df['origin'].unique() # array([1, 3, 2])
# 1. USA
# 2. EU
# 3. JAPAN

df['origin'] = df['origin'].replace({1 : 'USA',
                                     2 : 'EU',
                                     3 : 'JAPAN'})

df['origin'].unique()
# array(['USA', 'JAPAN', 'EU'], dtype=object)
df['origin'] = df['origin'].astype('category')

df.info()
'''
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   mpg           398 non-null    float64 
 1   cylinders     398 non-null    int64   
 2   displacement  398 non-null    float64 
 3   horsepower    398 non-null    object  
 4   weight        398 non-null    float64 
 5   acceleration  398 non-null    float64 
 6   model year    398 non-null    int64   
 7   origin        398 non-null    category
 8   name          398 non-null    object  
dtypes: category(1), float64(4), int64(2), object(2)
memory usage: 25.5+ KB
'''

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   name          398 non-null    object 
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   mpg           398 non-null    float64 
 1   cylinders     398 non-null    int64   
 2   displacement  398 non-null    float64 
 3   horsepower    398 non-null    o

In [ ]:
# 방금 한것처럼 범주형(category) 데이터로 변환하는 방법에
# 대해서 조금 더 알아보아요
# 일반적으로는 연속형 데이터를 그대로 사용하는것보다
# 범주형 데이터로 사용하는게 더 좋아요!(분석과 처리가 용이)

In [64]:
import numpy as np
import pandas as pd

df = pd.read_csv('./data/auto-mpg.csv',
                 header=None)

df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower',
              'weight', 'acceleration', 'model year', 'origin', 'name']

# horsepower : 마력(출력)
# 지금 이 horsepower가 실수로 되어 있는데 이걸 '저출력', '보통출력', '고출력'
# 이렇게 3개의 category로 변환
# horsepower안에 숫자가 아닌 ?가 들어가 있는게 있어요!

tmp = df['horsepower'].replace('?',
                               np.nan,
                               inplace=False)

df['horsepower'] = tmp
new_df = df.dropna(subset=['horsepower'],
                   axis=0,
                   inplace=False)

# 현재 object type으로 되어 있는데 숫자로 변경하면 될꺼 같아요
new_df['horsepower'] = new_df['horsepower'].astype(np.float64)

count, bin_dividers = np.histogram(new_df['horsepower'], bins=3)

print(count)            # [257 103  32]
print(bin_dividers)     # [ 46.         107.33333333 168.66666667 230.        ]

bin_names = ['저출력', '보통출력', '고출력']

# 컬럼을 새로 만들어요
new_df['hp_cut'] = pd.cut(x = new_df['horsepower'], # 구간화를 적용할 대상 데이터 (Series)
                          bins=bin_dividers,        # 구간 경계값 (히스토그램으로 구한 값)
                          labels=bin_names,         # 각 구간에 붙일 이름 (저출력, 보통출력, 고출력)
                          include_lowest=True)      # 경계값 포함여부 True이면 포함 False 미포함

display(new_df)

[257 103  32]
[ 46.         107.33333333 168.66666667 230.        ]


/tmp/ipykernel_1907/4130637726.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['horsepower'] = new_df['horsepower'].astype(np.float64)
/tmp/ipykernel_1907/4130637726.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['hp_cut'] = pd.cut(x = new_df['horsepower'], # 구간화를 적용할 대상 데이터 (Series)


,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,hp_cut
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,보통출력
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,보통출력
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,보통출력
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst,보통출력
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,보통출력
...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790.0,15.6,82,1,ford mustang gl,저출력
394,44.0,4,97.0,52.0,2130.0,24.6,82,2,vw pickup,저출력
395,32.0,4,135.0,84.0,2295.0,11.6,82,1,dodge rampage,저출력
396,28.0,4,120.0,79.0,2625.0,18.6,82,1,ford ranger,저출력


In [66]:
# DataFrame 결합
import numpy as np
import pandas as pd

df1 = pd.DataFrame({'a': ['a0', 'a1', 'a2', 'a3'],
                    'b': [1, 2, 3, 4],
                    'c': ['c0', 'c1', 'c2', 'c3']},
                   index=[0, 1, 2, 3])

df2 = pd.DataFrame({'b': ['b0', 'b1', 'b2', 'b3'],
                    'c': ['c0', 'c1', 'c2', 'c3'],
                    'd': ['d0', 'd1', 'd2', 'd3'],
                    'e': ['e0', 'e1', 'e2', 'e3']},
                   index=[2, 3, 4, 5])

display(df1, df2)

,a,b,c
0,a0,1,c0
1,a1,2,c1
2,a2,3,c2
3,a3,4,c3


,b,c,d,e
2,b0,c0,d0,e0
3,b1,c1,d1,e1
4,b2,c2,d2,e2
5,b3,c3,d3,e3


In [ ]:
# DataFrame 연결은 두가지 종류가 있어요
# 1. 그냥 이어붙이기
# 2. Database SQL에서 배운 JOIN

# 1. DataFrame 이어 붙이기
result_df = pd.concat([df1, df2], axis=0)
display(result_df)
'''
    a	b	c	d	e
0	a0	1	c0	NaN	NaN
1	a1	2	c1	NaN	NaN
2	a2	3	c2	NaN	NaN
3	a3	4	c3	NaN	NaN
2	NaN	b0	c0	d0	e0
3	NaN	b1	c1	d1	e1
4	NaN	b2	c2	d2	e2
5	NaN	b3	c3	d3	e3
'''

result_df1 = pd.concat([df1, df2], axis=1)
display(result_df1)
'''
    a	b	c	b	c	d	e
0	a0	1.0	c0	NaN	NaN	NaN	NaN
1	a1	2.0	c1	NaN	NaN	NaN	NaN
2	a2	3.0	c2	b0	c0	d0	e0
3	a3	4.0	c3	b1	c1	d1	e1
4	NaN	NaN	NaN	b2	c2	d2	e2
5	NaN	NaN	NaN	b3	c3	d3	e3
'''

,a,b,c,d,e
0,a0,1,c0,NaN,NaN
1,a1,2,c1,NaN,NaN
2,a2,3,c2,NaN,NaN
3,a3,4,c3,NaN,NaN
2,NaN,b0,c0,d0,e0
3,NaN,b1,c1,d1,e1
4,NaN,b2,c2,d2,e2
5,NaN,b3,c3,d3,e3


,a,b,c,b,c,d,e
0,a0,1.0,c0,NaN,NaN,NaN,NaN
1,a1,2.0,c1,NaN,NaN,NaN,NaN
2,a2,3.0,c2,b0,c0,d0,e0
3,a3,4.0,c3,b1,c1,d1,e1
4,NaN,NaN,NaN,b2,c2,d2,e2
5,NaN,NaN,NaN,b3,c3,d3,e3


In [116]:
# 2. Database SQL에서 배운 JOIN
# merge() 함수 사용
import numpy as np
import pandas as pd

data1 = { "학번" : [1,2,3,4],
          "이름" : ["홍길동","신사임당","이순신","강감찬"],
          "학년" : [2,4,1,3]}

data2 = { "학번" : [1,2,4,5],
          "학과" : ["CS","MATH","MATH","CS"],
          "학점" : [3.4,2.9,4.5,1.2]}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

result_df = pd.merge(df1, df2, on='학번', how='inner')
display(result_df)

result_df1 = pd.merge(df1, df2, on='학번', how='left')
display(result_df1)

result_df2 = pd.merge(df1, df2, on='학번', how='right')
display(result_df2)

result_df3 = pd.merge(df1, df2, on='학번', how='outer')
display(result_df3)

,학번,이름,학년,학과,학점
0,1,홍길동,2,CS,3.4
1,2,신사임당,4,MATH,2.9
2,4,강감찬,3,MATH,4.5


,학번,이름,학년,학과,학점
0,1,홍길동,2,CS,3.4
1,2,신사임당,4,MATH,2.9
2,3,이순신,1,NaN,NaN
3,4,강감찬,3,MATH,4.5


,학번,이름,학년,학과,학점
0,1,홍길동,2.0,CS,3.4
1,2,신사임당,4.0,MATH,2.9
2,4,강감찬,3.0,MATH,4.5
3,5,NaN,NaN,CS,1.2


,학번,이름,학년,학과,학점
0,1,홍길동,2.0,CS,3.4
1,2,신사임당,4.0,MATH,2.9
2,3,이순신,1.0,NaN,NaN
3,4,강감찬,3.0,MATH,4.5
4,5,NaN,NaN,CS,1.2


In [80]:
import numpy as np
import pandas as pd

data1 = { "학번" : [1,2,3,4],
          "이름" : ["홍길동","신사임당","이순신","강감찬"],
          "학년" : [2,4,1,3]}

data2 = { "학생학번" : [1,2,4,5],
          "학과" : ["CS","MATH","MATH","CS"],
          "학점" : [3.4,2.9,4.5,1.2]}


result_df = pd.merge(df1,
                     df2,
                     left_on='학번',
                     right_on='학생학번',
                     how='inner')

result_df

,학번,이름,학년,학생학번,학과,학점
0,1,홍길동,2,1,CS,3.4
1,2,신사임당,4,2,MATH,2.9
2,4,강감찬,3,4,MATH,4.5


In [ ]:
import numpy as np
import pandas as pd

data1 = { "학번" : [1,2,3,4],
          "이름" : ["홍길동","신사임당","이순신","강감찬"],
          "학년" : [2,4,1,3]}

data2 = { "학번" : [1,2,4,5],
          "학과" : ["CS","MATH","MATH","CS"],
          "학점" : [3.4,2.9,4.5,1.2]}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2,
                   index=[1,2,3,4,5])

display(df1, df2)
result_df = pd.merge(df1,
                     df2,
                     left_on='학번',
                     right_index=True,
                     how='inner')

display(result_df)


,학번,이름,학년
0,1,홍길동,2
1,2,신사임당,4
2,3,이순신,1
3,4,강감찬,3


,학번,학과,학점
1,1,CS,3.4
2,2,MATH,2.9
4,4,MATH,4.5
5,5,CS,1.2


,학번,학번_x,이름,학년,학번_y,학과,학점
0,1,1,홍길동,2,1,CS,3.4
1,2,2,신사임당,4,2,MATH,2.9
3,4,4,강감찬,3,4,MATH,4.5


In [93]:
import numpy as np
import pandas as pd

data1 = { "학번" : [1,2,3,4],
          "이름" : ["홍길동","신사임당","이순신","강감찬"],
          "학년" : [2,4,1,3]}

data2 = {"학과" : ["CS","MATH","MATH","CS"],
         "학점" : [3.4,2.9,4.5,1.2]}


df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2,
                    index=[1,2,4,5])


# display(df1,df2)

result_df = pd.merge(df1,
                     df2,
                     left_on='학번',
                     right_index= True,
                     how='inner'
                     )
result_df

# result_df = pd.merge(df1,
#                      df2,
#                      left_on='학번',
#                      right_on='학생학번',
#                      how='inner')
# result_df

,학번,이름,학년,학과,학점
0,1,홍길동,2,CS,3.4
1,2,신사임당,4,MATH,2.9
3,4,강감찬,3,MATH,4.5


In [ ]:
# Series에 함수 매핑
import seaborn as sns

titanic = sns.load_dataset('titanic')
display(titanic)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [ ]:
df = titanic.loc[:,['age', 'fare']]
display(df)

,age,fare
0,22.0,7.2500
1,38.0,71.2833
2,26.0,7.9250
3,35.0,53.1000
4,35.0,8.0500
...,...,...
886,27.0,13.0000
887,19.0,30.0000
888,NaN,23.4500
889,26.0,30.0000


In [89]:
def add_10(n):
    return n + 10

def add_a_b(n,a):
    return n + a + 10

print(df['age']) # 원래 나이

# age의 모든 행에 함수를 적용해서 각 사람의 나이를 10살 올리세요!
s1 = df['age'].apply(add_10) # Series 원소 각각에 함수적용
print(s1) # 10살씩 증가한 나이

s2 = df['age'].apply(add_a_b, a = 20) # 함수적용
print(s2)

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888     NaN
889    26.0
890    32.0
Name: age, Length: 891, dtype: float64
0      32.0
1      48.0
2      36.0
3      45.0
4      45.0
       ... 
886    37.0
887    29.0
888     NaN
889    36.0
890    42.0
Name: age, Length: 891, dtype: float64
0      52.0
1      68.0
2      56.0
3      65.0
4      65.0
       ... 
886    57.0
887    49.0
888     NaN
889    56.0
890    62.0
Name: age, Length: 891, dtype: float64


In [92]:
import seaborn as sns

titanic = sns.load_dataset('titanic')

df = titanic.loc[:,['age', 'fare']]

def add_10(n):
    return n + 10

# 이번에는 Series가 아니라 DataFrame에 각 요소에 함수를 적용
df2 = df.applymap(add_10)
display(df2)

/tmp/ipykernel_1907/1749070891.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2 = df.applymap(add_10)


,age,fare
0,32.0,17.2500
1,48.0,81.2833
2,36.0,17.9250
3,45.0,63.1000
4,45.0,18.0500
...,...,...
886,37.0,23.0000
887,29.0,40.0000
888,NaN,33.4500
889,36.0,40.0000


In [1]:
# Grouping

# 1. 하나의 열을 이용해서 Grouping
import seaborn as sns

titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age', 'sex', 'class', 'fare', 'survived']]
display(df)

# grouping을 해보아요
grouped = df.groupby('class', observed=True)
print(grouped)

# 사용하는 방법에 대해서 알아보자.
grouped[['age', 'fare']].mean() 

# 가장 간단한 사용법
for key, group in grouped:
    print(key)
    display(group)

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0
...,...,...,...,...,...
886,27.0,male,Second,13.0000,0
887,19.0,female,First,30.0000,1
888,NaN,female,Third,23.4500,0
889,26.0,male,First,30.0000,1


First


,age,sex,class,fare,survived
1,38.0,female,First,71.2833,1
3,35.0,female,First,53.1000,1
6,54.0,male,First,51.8625,0
11,58.0,female,First,26.5500,1
23,28.0,male,First,35.5000,1
...,...,...,...,...,...
871,47.0,female,First,52.5542,1
872,33.0,male,First,5.0000,0
879,56.0,female,First,83.1583,1
887,19.0,female,First,30.0000,1


Second


,age,sex,class,fare,survived
9,14.0,female,Second,30.0708,1
15,55.0,female,Second,16.0000,1
17,NaN,male,Second,13.0000,1
20,35.0,male,Second,26.0000,0
21,34.0,male,Second,13.0000,1
...,...,...,...,...,...
866,27.0,female,Second,13.8583,1
874,28.0,female,Second,24.0000,1
880,25.0,female,Second,26.0000,1
883,28.0,male,Second,10.5000,0


Third


,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
2,26.0,female,Third,7.9250,1
4,35.0,male,Third,8.0500,0
5,NaN,male,Third,8.4583,0
7,2.0,male,Third,21.0750,0
...,...,...,...,...,...
882,22.0,female,Third,10.5167,0
884,25.0,male,Third,7.0500,0
885,39.0,female,Third,29.1250,0
888,NaN,female,Third,23.4500,0


In [102]:
titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age', 'sex', 'class', 'fare', 'survived']]

grouped = df.groupby('class',
                     observed=True)

# 사용하는 방법에 대해서 알아보아요
grouped[['age', 'fare']].mean()
grouped['age'].max()

# 특정 그룹을 가져올 수 있어요
my_group = grouped.get_group('Third')
display(my_group)

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
2,26.0,female,Third,7.9250,1
4,35.0,male,Third,8.0500,0
5,NaN,male,Third,8.4583,0
7,2.0,male,Third,21.0750,0
...,...,...,...,...,...
882,22.0,female,Third,10.5167,0
884,25.0,male,Third,7.0500,0
885,39.0,female,Third,29.1250,0
888,NaN,female,Third,23.4500,0


In [ ]:
# 2개 이상의 열을 이용해서 Grouping 할꺼예요!

titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age', 'sex', 'class', 'fare', 'survived']]

grouped = df.groupby(['class', 'sex'],
                     observed=True)

for key, group in grouped:
    print(key)
    display(group.head(2))

my_group = grouped.get_group(('First', 'female'))
display(my_group)

('First', 'female')


,age,sex,class,fare,survived
1,38.0,female,First,71.2833,1
3,35.0,female,First,53.1000,1


('First', 'male')


,age,sex,class,fare,survived
6,54.0,male,First,51.8625,0
23,28.0,male,First,35.5000,1


('Second', 'female')


...

,age,sex,class,fare,survived
17,NaN,male,Second,13.0,1
20,35.0,male,Second,26.0,0


('Third', 'female')


,age,sex,class,fare,survived
2,26.0,female,Third,7.9250,1
8,27.0,female,Third,11.1333,1


('Third', 'male')


,age,sex,class,fare,survived
0,22.0,male,Third,7.25,0
4,35.0,male,Third,8.05,0


In [ ]:
titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age', 'sex', 'class', 'fare', 'survived']]

grouped = df.groupby(['class', 'sex'],
                     observed=True)

# 'First', 'Second', 'Third' 3개의 group
print(grouped['fare'].mean())
'''
class   sex   
First   female    106.125798
        male       67.226127
Second  female     21.970121
        male       19.741782
Third   female     16.118810
        male       12.661633
Name: fare, dtype: float64
'''
print(grouped.fare.mean())
'''
class   sex   
First   female    106.125798
        male       67.226127
Second  female     21.970121
        male       19.741782
Third   female     16.118810
        male       12.661633
Name: fare, dtype: float64
'''
print(grouped.survived.mean())
'''
class   sex   
First   female    0.968085
        male      0.368852
Second  female    0.921053
        male      0.157407
Third   female    0.500000
        male      0.135447
Name: survived, dtype: float64
'''

# 사용자 정의 함수를 만들어서 사용
def minmax(x):
    return x.max() - x.min()

print(grouped.fare.agg(minmax))
'''
class   sex   
First   female    486.4000
        male      512.3292
Second  female     54.5000
        male       73.5000
Third   female     62.8000
        male       69.5500
Name: fare, dtype: float64
'''

class   sex   
First   female    106.125798
        male       67.226127
Second  female     21.970121
        male       19.741782
Third   female     16.118810
        male       12.661633
Name: fare, dtype: float64
class   sex   
First   female    106.125798
        male       67.226127
Second  female     21.970121
        male       19.741782
Third   female     16.118810
        male       12.661633
Name: fare, dtype: float64
class   sex   
First   female    0.968085
        male      0.368852
Second  female    0.921053
        male      0.157407
Third   female    0.500000
        male      0.135447
Name: survived, dtype: float64
class   sex   
First   female    486.4000
        male      512.3292
Second  female     54.5000
        male       73.5000
Third   female     62.8000
        male       69.5500
Name: fare, dtype: float64


In [ ]:
# 기본적인 grouping방법에 대해서 알아보았는데
# 추가적으로 1가지만 더 알아보아요!
# 그룹에 대한 조건을 설정할 수 있어요! => filter

titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age', 'sex', 'class', 'fare', 'survived']]

grouped = df.groupby('class',
                     observed=True)

display(grouped.fare.count())
'''
class
First     216
Second    184
Third     491
Name: fare, dtype: int64
'''

# 데이터가 300개 이상의 그룹만을 필터링 하고 싶어요!
filter_group = grouped.filter(lambda x: len(x) >= 300)
display(filter_group)

class
First     216
Second    184
Third     491
Name: fare, dtype: int64

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
2,26.0,female,Third,7.9250,1
4,35.0,male,Third,8.0500,0
5,NaN,male,Third,8.4583,0
7,2.0,male,Third,21.0750,0
...,...,...,...,...,...
882,22.0,female,Third,10.5167,0
884,25.0,male,Third,7.0500,0
885,39.0,female,Third,29.1250,0
888,NaN,female,Third,23.4500,0


'\nage\tsex\tclass\tfare\tsurvived\n0\t22.0\tmale\tThird\t7.2500\t0\n2\t26.0\tfemale\tThird\t7.9250\t1\n4\t35.0\tmale\tThird\t8.0500\t0\n5\tNaN\tmale\tThird\t8.4583\t0\n7\t2.0\tmale\tThird\t21.0750\t0\n...\t...\t...\t...\t...\t...\n882\t22.0\tfemale\tThird\t10.5167\t0\n884\t25.0\tmale\tThird\t7.0500\t0\n885\t39.0\tfemale\tThird\t29.1250\t0\n888\tNaN\tfemale\tThird\t23.4500\t0\n890\t32.0\tmale\tThird\t7.7500\t0\n491 rows × 5 columns\n'